In [ ]:
# !pip install openpyxl
# !pip install scikit-learn

In [1]:
import pandas as pd
import numpy as np

In [2]:
# https://data.mendeley.com/datasets/9jhzmswdfj/1 
# df = pd.read_excel("XSS_dataset.xlsx")

file_path = "https://drive.google.com/uc?export=download&id=1AFEgzMFqggjE0UCPiEJ4Utf5hMJWQ6MA"
df = pd.read_excel(file_path)
df.head()

,App Names,Permissions,API Name,Website Name,IP,Location,Label
0,XSSDetector,"INTERNET,READ_CONTACTS,ACCESS_FINE_LOCATION,AC...",getDeviceId,http://www.5riversmarketing.com/,104.243.40.3,USA - Florida,Yes
1,Attackers 2,"INTERNET,READ_CONTACTS,ACCESS_FINE_LOCATION,AC...",getSimSerialNumber,http://www.5riversmarketing.com/,104.243.40.3,USA - Florida,Yes
2,Attackers 3,"INTERNET,ACCESS_FINE_LOCATION,ACCESS_COARSE_LO...",Not Sensitive,http://www.5riversmarketing.com/,104.243.40.3,USA - Florida,No
3,Attackers 4,"INTERNET,READ_PHONE_STATE,SEND_SMS",Not Sensitive,http://demo2.5riversmarketing.com/,104.243.40.3,USA - Florida,No
4,Attackers 5,"INTERNET,READ_CONTACTS,READ_PHONE_STATE",Not Sensitive,http://www.5riversmarketing.com/,104.243.40.3,USA - Florida,No


In [ ]:
# print("Unique App Names: ", len(pd.unique(df['App Names'])))
# print("Unique Permissions: ", len(pd.unique(df['Permissions'])))
# print("Unique API Names: ", len(pd.unique(df['API Name'])))
# print("Unique Website Names: ", len(pd.unique(df['Website Name'])))
# print("Unique IP: ", len(pd.unique(df['IP'])))
# print("Unique Locations: ", len(pd.unique(df['Location'])))
# print("Unique Label: ", len(pd.unique(df['Label'])))

In [ ]:
df.dtypes

In [3]:
# maps = {}
def encode_strings_to_numeric(df):
    encoded_df = pd.DataFrame()
    index = 0
    for col in df.columns:
        if df[col].dtype == 'object':  # Checking if the column contains string/object data
            unique_values = df[col].unique()  # Get unique values in the column
            mapping = {value: i for i, value in enumerate(unique_values)}  # Create a mapping of unique values to numeric values
            encoded_df[col] = df[col].map(mapping).fillna(df[col])  # Map string values to numeric using the mapping, keeping non-string values as is
        else:
            encoded_df[col] = df[col]  # If not a string column, keep the column as is
        # maps[index] = mapping
        index = index + 1

    return encoded_df

# Replace 'df' with your dataframe variable
# Call the function to encode string data to numeric form
encoded_df = encode_strings_to_numeric(df)
df = encoded_df

In [4]:
encoded_df.to_csv("new.csv")

In [5]:
df.tail()

,App Names,Permissions,API Name,Website Name,IP,Location,Label
460,31,14,15,117,118,95,0
461,32,9,2,118,119,96,0
462,33,10,16,119,120,97,0
463,34,15,17,120,121,98,0
464,35,16,18,121,122,99,0


In [6]:
# check correlation of all attributes with target atttribute
df.corr().abs()['Label'].sort_values(ascending = False)

# API Name has highest correlation with Label

Label           1.000000
API Name        0.746306
Permissions     0.131816
Website Name    0.020362
Location        0.020211
IP              0.019968
App Names       0.011751
Name: Label, dtype: float64

In [7]:
Y = df['Label']
X = df.drop(['Label'], axis = 1)

In [8]:
from numpy import mean, std

from sklearn.datasets import make_classification

from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler # to standardize dataset

from sklearn.pipeline import make_pipeline

In [9]:
def pipelined_models():
    models = dict()
    
    models['logistic regression'] = make_pipeline(StandardScaler(), LogisticRegression())
    models['KNeighborsClassifier'] = make_pipeline(StandardScaler(), KNeighborsClassifier())
    models['DecisionTreeClassifier'] = make_pipeline(StandardScaler(), DecisionTreeClassifier())
    models['SVC'] = make_pipeline(StandardScaler(), SVC())
    models['GaussianNB'] = make_pipeline(StandardScaler(), GaussianNB())
    
    return models

models = pipelined_models()

In [10]:
def model_evaluation(model, X, Y):
    # cross_val : split data for both train and test to evaluate
    # split k = 10 equal parts and train k - 1 parts and test on remaining
    # stratified = same proportion of classes in each fold / part
    # random state = consistency
    cross_validation = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 1)
    score_list = cross_val_score(model, X, Y, scoring = 'accuracy', cv = cross_validation, n_jobs = -1, error_score='raise')
    return score_list

In [11]:
res = list()
names = list()

for name, model in models.items():
    scores = model_evaluation(model, X, Y)
    res.append(scores)
    names.append(name)
    print('%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

logistic regression 0.929 (0.039)
KNeighborsClassifier 0.876 (0.047)
DecisionTreeClassifier 0.996 (0.010)
SVC 0.917 (0.036)
GaussianNB 0.998 (0.006)


In [ ]:
# only to compare
import matplotlib.pyplot as plt
plt.boxplot(res, labels = names, showmeans = True)
plt.show()

In [ ]:
# Accuracy:
    # Decision Tree: 99.6%
    # Gaussian Naive Bayes: 99.8%
    # Logistic regression: 92.9%
    # KNeighborsClassifier 87.6%
    # SVC 91.6%

In [12]:
from sklearn.model_selection import train_test_split
X = df.drop(columns = 'Label')
Y = df['Label']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 42)

In [23]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = XGBClassifier(random_state = 42)
model.fit(x_train, y_train)
y_val_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_val_pred)
print(f"XGBoost Validation Accuracy(XGBoost): {accuracy}")

XGBoost Validation Accuracy(XGBoost): 0.9928571428571429


In [24]:
# Accuracy for XGBoost: 99.2%, but for Decision Tree: 99.8%
model = models['GaussianNB']

#fitting the model
model.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gaussiannb', GaussianNB())])

In [25]:
def encode_ip(ip_data):
    ip_col = ['App Names', 'Permissions', 'API Name', 'Website Name', 'IP', 'Location']
    encoded_data = []
    i = 0
    for data in ip_data:
        col = ip_col[i]
        rows = df.index[df[col] == data].tolist()
        row = 0
        if(len(rows) != 0):
            row = rows[0]
        val = encoded_df.loc[row, col]
        encoded_data.append(val)
        i = i + 1
    return encoded_data

In [26]:
ip_data = ['SQUIRREL', 'DISABLE_KEYGUARD,EXPAND_STATUS_BAR,GET_PACKAGE_SIZE,INSTALL_SHORTCUT,INTERNET,KILL_BACKGROUND_PROCESSES,MANAGE_OWN_CALLS,MODIFY_AUDIO_SETTINGS,NFC,READ_SYNC_SETTINGS,READ_SYNC_STATS', 'Not Sensitive', 'https://www.bloomberg.com/', '109.8.27.15', 'France']
encoded_data = encode_ip(ip_data)

In [27]:
ip_data_np = np.asarray(encoded_data)
ip_data_reshape = ip_data_np.reshape(1, -1)
pred = model.predict(ip_data_reshape)
# print(pred)

if pred[0] == 0:
    print("XSS attack on the website.")
else:
    print("No XSS attack on the website.")

XSS attack on the website.


In [28]:
import pickle

In [29]:
filename = "xss_attack.sav"
pickle.dump(model, open(filename, "wb"))

In [31]:
ip_data = ['Attackers 4', 'INTERNET,READ_PHONE_STATE,SEND_SMS', 'Not Sensitive', 'http://demo2.5riversmarketing.com/', '104.243.40.3', 'USA - Florida']

encoded_data = encode_ip(ip_data)
    
# print(encoded_data)
ip_data_np = np.asarray(encoded_data)
ip_data_reshape = ip_data_np.reshape(1, -1)
pred = model.predict(ip_data_reshape)
# print(pred)

if pred[0] == 0:
    print("XSS attack on the website.")
else:
    print("No XSS attack on the website.")


XSS attack on the website.


In [22]:
import warnings

# Suppress the specific warning about feature names
warnings.filterwarnings("ignore", message="X does not have valid feature names, but StandardScaler was fitted with feature names", category=UserWarning)
